In [2]:
# user interface demo
# core functionality:
#   midi file to sheet music
#   creating the structure for markov's chain
#   predicting next note based on last note
# convert back to midi file
# output and download

# unit tests
# test coverage
# code quality

Imports

In [ ]:
from mido import MidiFile, MidiTrack, Message
import random
import os


Converting a midi number to pitch

In [4]:
def note(midi_note):
    notes = [
        'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'
    ]
    octave = (midi_note // 12) - 2  
    note_name = notes[midi_note % 12]
    return f"{note_name}{octave}"

Converting pitch to midi number

In [5]:
def midi(note):
    notes = [
        'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'
    ]
    i=0
    while notes[i]!=note[:-1]:
        i+=1
        
    return (int(note[-1])+2) * 12 + i

Converting midi to pitch, duration

In [6]:
def midi_to_pitch(file):
    midi_file = MidiFile(file)
    pitches = []
    duration = []
    midi_pitches = []
    for track in midi_file.tracks:
        for msg in track:
            if msg.type == 'note_on':  # note_on event
                #pitch = midi_to_pitch(msg.note)  # Convert MIDI number to pitch name
                pitches.append(note(msg.note))
                midi_pitches.append(msg.note)
            if msg.type == 'note_off':  # note_on event
                duration.append(msg.time)
            # if msg.type == 'program_change':
            #     print(msg.program)
    return pitches, duration

Converting pitch, duration to midi

In [7]:
def pitch_to_midi(pitches, instrument):
    notes = []
    # Create a new MIDI file
    midi_file = MidiFile()
    track = MidiTrack()
    midi_file.tracks.append(track)
    track.append(Message('program_change', program=instrument))
    for i in range(len(pitches)):
        notes.append((midi(pitches[i]), 300))
        #notes.append((midi(pitches[i]), duration[i]))
    for pitch, d in notes:
        track.append(Message('note_on', note=pitch, velocity=64, time=0))  # Start the note
        track.append(Message('note_off', note=pitch, velocity=64, time=d))  # Stop the note after 'duration' ticks

    midi_file.save('output.mid')

In [8]:
class Node:
    def __init__(self, note):
        self.note = note
        self.probability = 0
        self.freq = 1
        self.children = {}
    
    def add_child(self, note):
        self.children[note] = Node(note)

    
    def new_notes(self, note2, note3):
        self.freq += 1
        if note2 in self.children:
            for child in self.children:
                if child == note2:
                    self.children[child].freq += 1
                    if note3 in self.children[child].children:
                        for child2 in self.children[child].children:
                            if child2 == note3:
                                self.children[child].children[child2].freq += 1
                    else:
                        self.children[child].add_child(note3)
        else:
            self.add_child(note2)
            self.children[note2].add_child(note3)

In [9]:
def get_prob():
    for root in roots.values():
        for node2 in root.children:
            root.children[node2].probability = root.children[node2].freq/root.freq
            for node3 in root.children[node2].children:
                root.children[node2].children[node3].probability = root.children[node2].children[node3].freq/root.children[node2].freq


In [10]:
def print_trie():
    for root in roots.values():
        print(root.note)
        for node2 in root.children:
            print("\t", node2, round(root.children[node2].probability, 2))
            for node3 in root.children[node2].children:
                print("\t\t", node3, round(root.children[node2].children[node3].probability, 2))

        print("\n \n")

Creating the probability trie

In [11]:
#pitches = ["C", "A", "D", "C", "B", "A", "C", "A", "E", "C", "B", "E", "C", "B", "A"]
roots = {}
def trie(pitches, duration):
    for i in range(len(pitches)-2):
        if pitches[i] not in roots:
            roots[pitches[i]] = Node(pitches[i])
            roots[pitches[i]].freq -= 1
        roots[pitches[i]].new_notes(pitches[i+1], pitches[i+2])
    get_prob()

In [12]:
import os

current_directory = os.getcwd()
print(current_directory)

C:\Users\arina\Documents\GitHub\AI-Lab


In [ ]:
directory = 'MIDI'
files = os.listdir(directory)
for file in files:
    

['marche_funebre.mid', 'mazurque.mid', 'nutcracker.mid', 'sonata13.mid', 'sonata22.mid', 'sonata9.mid', 'vivaldi_autumn.mid', 'vivaldi_spring.mid', 'vivaldi_summer.mid', 'vivaldi_winter.mid']


In [14]:
pitches, duration = midi_to_pitch('MIDI/vivaldi_spring.mid')
trie(pitches, duration)
pitches, duration = midi_to_pitch('MIDI/vivaldi_summer.mid')
trie(pitches, duration)
pitches, duration = midi_to_pitch('MIDI/vivaldi_winter.mid')
trie(pitches, duration)
pitches, duration = midi_to_pitch('MIDI/vivaldi_autumn.mid')
trie(pitches, duration)
pitches, duration = midi_to_pitch('MIDI/sonata13.mid')
trie(pitches, duration)
pitches, duration = midi_to_pitch('MIDI/marche_funebre.mid')
trie(pitches, duration)
pitches, duration = midi_to_pitch('MIDI/mazurque.mid')
trie(pitches, duration)
pitches, duration = midi_to_pitch('MIDI/nutcracker.mid')
trie(pitches, duration)
pitches, duration = midi_to_pitch('MIDI/sonata9.mid')
trie(pitches, duration)
pitches, duration = midi_to_pitch('MIDI/sonata22.mid')
trie(pitches, duration)

In [15]:
print_trie()

C#3
	 E3 0.09
		 G#3 0.32
		 A3 0.1
		 G3 0.03
		 B2 0.04
		 F3 0.02
		 D3 0.03
		 A2 0.06
		 F#3 0.02
		 C#3 0.03
		 E3 0.19
		 D#3 0.02
		 D2 0.01
		 E4 0.02
		 C#2 0.01
		 G2 0.01
		 C#4 0.09
		 F2 0.01
	 F#3 0.04
		 A3 0.16
		 F#3 0.68
		 C#4 0.05
		 E3 0.03
		 F#2 0.01
		 C#3 0.01
		 G#3 0.03
		 D#3 0.01
		 B2 0.01
	 G3 0.02
		 A#3 0.13
		 D#1 0.02
		 D#3 0.02
		 G#3 0.02
		 C#4 0.07
		 G3 0.07
		 F#3 0.36
		 C#3 0.02
		 A2 0.02
		 A3 0.04
		 F3 0.04
		 C3 0.16
		 G#2 0.02
	 D#3 0.07
		 C#3 0.41
		 C#4 0.03
		 D#3 0.24
		 G2 0.02
		 A#2 0.03
		 A2 0.01
		 G#2 0.02
		 F2 0.01
		 D#2 0.01
		 G3 0.02
		 B3 0.02
		 D#4 0.02
		 F#3 0.03
		 E3 0.06
		 B2 0.02
		 C3 0.04
		 F3 0.02
	 C#3 0.36
		 C#3 0.26
		 G#2 0.02
		 F#3 0.07
		 D#3 0.04
		 A2 0.02
		 G#3 0.01
		 D3 0.05
		 C4 0.0
		 F3 0.0
		 C#4 0.14
		 D#2 0.0
		 B2 0.08
		 G2 0.04
		 A#2 0.02
		 C#2 0.02
		 C3 0.01
		 A#3 0.01
		 G3 0.04
		 G#1 0.01
		 E3 0.07
		 A3 0.02
		 E2 0.02
		 F#2 0.02
		 E4 0.0
		 A4 0.0
		 D2 0.02
		 F2 0

In [16]:
input = ["C#3", "E3"]
melody = input.copy()

In [24]:
for i in range(50):
    note1 = melody[i]
    note2 = melody[i+1]
    node2 = roots[note1].children[note2]
    dist = list(node2.children.keys())
    weights = list([j.probability*100 for j in node2.children.values()])
    pred_note = random.choices(dist, weights = weights, k = 1)[0]
    melody.append(pred_note)
    #print(pred_note)
print(melody)

['C#3', 'E3', 'E3', 'C3', 'E3', 'G3', 'C#4', 'F#3', 'F#3', 'E3', 'E3', 'D3', 'A2', 'A2', 'F#2', 'F#2', 'A2', 'G2', 'A1', 'A1', 'A1', 'G1', 'F#1', 'E1', 'D#1', 'C#1', 'B0', 'C1', 'C#1', 'B0', 'C1', 'C1', 'C1', 'C1', 'C1', 'C2', 'A0', 'A1', 'A#0', 'A#1', 'A#1', 'A#1', 'A#1', 'D#1', 'G#1', 'G#1', 'G#1', 'G#1', 'G#1', 'D#1', 'A#1', 'A#1', 'G#3', 'C#4', 'C3', 'A2', 'G3', 'D3', 'F#3', 'A3', 'F#3', 'G3', 'D3', 'A2', 'F#3', 'D2', 'F#2', 'G2', 'D#2', 'A1', 'A1', 'A1', 'G1', 'F#1', 'D#1', 'D#1', 'B0', 'C1', 'C#1', 'C1', 'B0', 'C#1', 'C#1', 'C1', 'C#1', 'C1', 'C1', 'A1', 'B0', 'A#1', 'D#1', 'A#2', 'D#1', 'A#1', 'G#1', 'B1', 'D#2', 'G#1', 'G#0', 'E2', 'A#1', 'D#2', 'E3', 'C#3', 'C3', 'G3', 'E4', 'G3', 'C#3', 'D#4', 'E3', 'E3', 'A2', 'F#2', 'D2', 'F#2', 'B2', 'A2', 'D#2', 'F#2', 'G#1', 'A1', 'F1', 'F#2', 'D#1', 'C#1', 'B0', 'C1', 'C#1', 'B0', 'C1', 'C2', 'C1', 'F1', 'C2', 'C#1', 'C2', 'A1', 'G#0', 'A#1', 'D#1', 'A#2', 'D#1', 'A#1', 'G#1', 'D#2', 'B1', 'A#1', 'F#1', 'A1', 'D#1', 'D#2', 'G#3', 'G2', 

In [25]:
instrument = 48
pitch_to_midi(melody, instrument)